<center>
    <h1> Scientific Programming in Python  </h1>
    <h2> Topic 4: Just in Time Compilation: Numba and NumExpr </h2> 
</center>

In [1]:
import numba
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

En esta actividad implementaremos una conocida métrica para medir disimilitud entre conjuntos: __La métrica de Hausdorff__. Esta corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 


Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.
<img src='data/hausdorff.png' style="width: 600px;">

1. Implemente la métrica de Hausdorff en Python clásico.
2. Implemente la métrica de Hausdorff usando Numba (Forzando el modo __nopython__ y definiendo explícitamente las _signatures_ de las funciones).
3. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las funciones anteriores sobre estos arreglos.
4. Concluya.

In [2]:
def dist(a, b):
    return np.sqrt((a - b)**2)

def max_min_dist(X, Y):
    m,n = X.shape
    maxs = np.empty(m)
    for k in np.arange(m):
        mins = np.empty((m,n))
        for i in np.arange(m):
            for j in np.arange(n):
                mins[i,j] = dist(X[k,j], Y[i,j])
        maxs[k] = np.amin(mins[k,:])
    return np.amax(maxs)   
    
def hausdorff(X, Y):
    return np.maximum(max_min_dist(X, Y), max_min_dist(Y, X))

In [3]:
@numba.jit('float64 (float64, float64)',nopython=True)
def numba_dist(a, b):
    return np.sqrt((a - b)**2)

@numba.jit('float64 (float64[:,:], float64[:,:])',nopython=True)
def numba_max_min_dist(X, Y):
    m,n = X.shape
    maxs = np.empty(m)
    for k in np.arange(m):
        mins = np.empty((m,n))
        for i in np.arange(m):
            for j in np.arange(n):
                mins[i,j] = numba_dist(X[k,j], Y[i,j])
        maxs[k] = np.amin(mins[k,:])
    return np.amax(maxs)    

@numba.jit('float64 (float64[:,:], float64[:,:])',nopython=True)
def numba_hausdorff(X, Y):
    return np.maximum(numba_max_min_dist(X, Y), numba_max_min_dist(Y, X))

In [ ]:
X1 = np.random.random((10,10))
Y1 = np.random.random((10,10))

X2 = np.random.random((50,50))
Y2 = np.random.random((50,50))

X3 = np.random.random((100,100))
Y3 = np.random.random((100,100))

X4 = np.random.random((200,200))
Y4 = np.random.random((200,200))

X5 = np.random.random((300,300))
Y5 = np.random.random((300,300))

X6 = np.random.random((400,400))
Y6 = np.random.random((400,400))

X7 = np.random.random((500,500))
Y7 = np.random.random((500,500))

X8 = np.random.random((1000,1000))
Y8 = np.random.random((1000,1000))

X9 = np.random.random((1500,1500))
Y9 = np.random.random((1500,1500))

X10 = np.random.random((2000,2000))
Y10 = np.random.random((2000,2000))

In [ ]:
%%timeit
hausdorff(X1, Y1)
hausdorff(X2, Y2)
hausdorff(X3, Y3)
hausdorff(X4, Y4)
hausdorff(X5, Y5)
hausdorff(X6, Y6)
hausdorff(X7, Y7)
hausdorff(X8, Y8)
hausdorff(X9, Y9)
hausdorff(X10, Y10)

In [ ]:
%%timeit
numba_hausdorff(X1, Y1)
numba_hausdorff(X2, Y2)
numba_hausdorff(X3, Y3)
numba_hausdorff(X4, Y4)
numba_hausdorff(X5, Y5)
numba_hausdorff(X6, Y6)
numba_hausdorff(X7, Y7)
numba_hausdorff(X8, Y8)
numba_hausdorff(X9, Y9)
numba_hausdorff(X10, Y10)

Podemos concluir que blabla